In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, FloatType, TimestampType, StructType, StructField
#sc.install_pypi_package('pip==22.0.4')
#sc.uninstall_package('matplotlib')
#sc.install_pypi_package("matplotlib==3.2.1")
#sc.install_pypi_package("numpy")





#this example has data from 2009 -2019 partitioned by year/month. To load multple partitions at once - can use
# glob expression like below. This loads in data from years 2017-2019

# vendor_id — a code indicating the provider associated with the trip record
# pickup_datetime — date and time when the meter was engaged
# dropoff_datetime — date and time when the meter was disengaged
# passenger_count — the number of passengers in the vehicle (driver entered value)
# store_and_fwd_flag — This flag indicates whether the trip record was held in vehicle memory before sending to the vendor
#because the vehicle did not have a connection to the server — Y=store and forward; N=not a store and forward trip.
#trip_duration — duration of the trip in seconds



schema = StructType([StructField("vendor_id",StringType(),False),\
             StructField("pickup_at",TimestampType(),True),\
             StructField("dropoff_at",TimestampType(),True),\
             StructField("passenger_count",IntegerType(),True),\
             StructField("trip_distance",FloatType(),True),\
             StructField("rate_code_id",StringType(),True),\
             StructField("store_and_fwd_flag",StringType(),True),\
             StructField("pickup_location_id",IntegerType(),True),\
             StructField("dropoff_location_id",IntegerType(),True),\
             StructField("payment_type",StringType(),True),\
             StructField("fare_amount",FloatType(),True),\
             StructField("extra",FloatType(),True),\
             StructField("mta_tax",FloatType(),True),\
             StructField("tip_amount",FloatType(),True),\
             StructField("tolls_amount",FloatType(),True),\
             StructField("improvement_surcharge",FloatType(),True),\
             StructField("total_amount",FloatType(),True)])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1651428963520_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-cfn-bootstrap (2.0)
beautifulsoup4 (4.9.1)
boto (2.49.0)
click (7.1.2)
docutils (0.14)
jmespath (0.10.0)
joblib (0.15.1)
lockfile (0.11.0)
lxml (4.5.1)
mysqlclient (1.4.2)
nltk (3.5)
nose (1.3.4)
numpy (1.16.5)
pip (9.0.1)
py-dateutil (2.2)
pystache (0.5.4)
python-daemon (2.2.3)
python37-sagemaker-pyspark (1.3.0)
pytz (2020.1)
PyYAML (5.3.1)
regex (2020.6.8)
setuptools (28.8.0)
simplejson (3.2.0)
six (1.13.0)
soupsieve (1.9.5)
tqdm (4.46.1)
wheel (0.29.0)
windmill (1.6)

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [12]:


#if results already saved previoiusly in parquet - 
#df = spark.read.parquet('s3://big-datasets-over-gb/taxi_nyc/output_transformed_df.parquet')
#lets persist this -  note only persists if followed by action

#from pyspark import StorageLevel

#df.persist(StorageLevel.MEMORY_AND_DISK_2).count()

#df.unpersist()

#df.storageLevel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StorageLevel(True, True, False, False, 2)

In [29]:

#Adaptive Query Execution (AQE) is one of the greatest features of Spark 3.0 which reoptimizes and adjusts query plans based on runtime statistics collected during the execution of the query.

# Prior to 3.0, Spark does the optimization by creating an execution plan before the query starts executing, once execution starts Spark doesn’t do any further optimization which is based on metrics it collects during runtime. AQE bridges this gap by applying the second level of optimization based on the metrics it sees with each stage.

# Adaptive Query Execution is disabled by default. In order to enable Set spark.sql.adaptive.enabled configuration property to true.

# After enabling Adaptive Query Execution, Spark performs 2x improvement on TPC-DS over Spark 2.4 

# Spark 3.0 comes with three major features in AQE.

# Coalescing Post-shuffle Partitions that dynamically determine the optimal number of partitions
# Converting sort-merge join to Broadcast join, and
# Skew Join Optimization

spark.conf.set("spark.sql.adaptive.enabled", True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
## check existing spark conf

print(f"executor memory: {spark.conf.get('spark.executor.memory')}")
print(f"executor cores: {spark.conf.get('spark.executor.cores')}")
print(f"driver memory: {spark.conf.get('spark.driver.memory')}")
#The resources are adjusted dynamically based on the workload. 
#The app will give resources back if you are no longer using it.
print(f"Dynamic Allocation: {spark.conf.get('spark.dynamicAllocation.enabled')}")
print(f"Adaptive Query Execution: {spark.conf.get('spark.sql.adaptive.enabled')}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

executor memory: 9108M
executor cores: 4
driver memory: 2048M
Dynamic Allocation: true
Adaptive Query Execution: true

In [88]:
data = spark.\
read.\
option("header", "true").\
option("mergeSchema", "true").\
schema(schema).\
parquet("s3://ursa-labs-taxi-data/2017/**/*.parquet",\
        "s3://ursa-labs-taxi-data/2018/**/*.parquet",\
        "s3://ursa-labs-taxi-data/2019/**/*.parquet")

data.tail(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(vendor_id='2', pickup_at=datetime.datetime(2018, 10, 31, 23, 58, 45), dropoff_at=datetime.datetime(2018, 11, 1, 0, 11, 23), passenger_count=1, trip_distance=2.559999942779541, rate_code_id='1', store_and_fwd_flag='N', pickup_location_id=236, dropoff_location_id=163, payment_type='1', fare_amount=11.5, extra=0.5, mta_tax=0.5, tip_amount=2.559999942779541, tolls_amount=0.0, improvement_surcharge=0.30000001192092896, total_amount=15.359999656677246), Row(vendor_id='2', pickup_at=datetime.datetime(2018, 10, 31, 23, 8, 2), dropoff_at=datetime.datetime(2018, 10, 31, 23, 8, 7), passenger_count=1, trip_distance=0.0, rate_code_id='1', store_and_fwd_flag='N', pickup_location_id=193, dropoff_location_id=193, payment_type='2', fare_amount=0.0, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.0, total_amount=0.0)]

In [89]:
# we have over 319 million rows
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

319105374

In [90]:
data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['vendor_id', 'pickup_at', 'dropoff_at', 'passenger_count', 'trip_distance', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

In [91]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_at: timestamp (nullable = true)
 |-- dropoff_at: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- rate_code_id: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)

In [92]:
#simple example of udf
from pyspark.sql.functions import col
# .withColumn('extras', data.extra + data.mta_tax + data.tip_amount + data.tolls_amount + data.improvement_surcharge)\

F.udf(returnType=FloatType())


def time_diff(a, b):
    """
    Time difference in minutes from taxi pickup and drop off
    a: pickup start time
    b: pickup end time
    """
    #return F.col(a).cast("long") - F.col(b).cast("long")
    mins = (F.unix_timestamp(b) - F.unix_timestamp(a)) / 60
    return F.round(mins)


F.udf(returnType=FloatType())


def sum_extras():
    return col('extra') + col('mta_tax') + col('tip_amount') + col(
        'tolls_amount') + col('improvement_surcharge')


df = data.withColumn('duration', time_diff('pickup_at', 'dropoff_at'))\
.withColumn('extras', sum_extras())\
.withColumn('month', F.month('pickup_at'))\
.withColumn('year', F.year('pickup_at'))\
.withColumnRenamed('trip_distance', 'distance')\
.withColumnRenamed('fare_amount', 'fare')\
.withColumnRenamed('total_amount', 'total_cost')\
.dropDuplicates()\
.filter("duration >= 0 and total_cost >= 0")\
.select('vendor_id','month','year','duration','distance','fare','total_cost','extras','payment_type','passenger_count')


df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+----+--------+--------+----+----------+---------+------------+---------------+
|vendor_id|month|year|duration|distance|fare|total_cost|extras   |payment_type|passenger_count|
+---------+-----+----+--------+--------+----+----------+---------+------------+---------------+
|2        |9    |2017|0.0     |0.0     |90.0|95.3      |5.3      |1           |2              |
|2        |9    |2017|3.0     |0.68    |4.5 |5.3       |0.8      |1           |1              |
|2        |9    |2017|3.0     |0.65    |4.5 |7.3       |2.8      |1           |1              |
|1        |9    |2017|4.0     |0.6     |4.5 |6.95      |2.45     |1           |1              |
|2        |9    |2017|5.0     |0.87    |5.5 |6.3       |0.8      |2           |1              |
|2        |9    |2017|5.0     |0.89    |6.0 |7.8       |1.8      |2           |3              |
|1        |9    |2017|6.0     |1.0     |6.5 |9.35      |2.85     |1           |1              |
|1        |9    |2017|8.0     |1.9     |

In [93]:
# saving intermediate results to parquet to load in later

#df.write.format('parquet').mode('overwrite').partitionBy("month").save('s3://big-datasets-over-gb/taxi_nyc/output_transformed_df.parquet')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# each partition has just over a million rows.

print(
    f"Dataframe has {int(df.count()/1000000)} million rows and {df.rdd.getNumPartitions()} partitions"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dataframe has 216 million rows and 200 partitions

In [8]:
# which vendor made the most money in trips

df.groupBy('vendor_id').\
agg((F.sum('total_cost')/F.lit(1000000)).cast(IntegerType()).alias('Total_Earnings(millions)'),\
    F.round(F.mean('total_cost'),1).alias('Average_Earnings($)'),\
    F.round(F.mean('extras'),1).alias('Average_Extras($)'),\
    F.round(F.mean('distance'),1).alias('Average_Distance'),\
    F.round(F.mean('duration'),1).alias('Average_Duration(mins)'),\
    F.round(F.count(col('fare'))/F.lit(1000000), 1).alias('Number_of_Trips(Millions)')).\
orderBy([col('Average_Earnings($)'),col('Average_Distance')], ascending=[0,0]).\
show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------------+-------------------+-----------------+----------------+----------------------+-------------------------+
|vendor_id|Total_Earnings(millions)|Average_Earnings($)|Average_Extras($)|Average_Distance|Average_Duration(mins)|Number_of_Trips(Millions)|
+---------+------------------------+-------------------+-----------------+----------------+----------------------+-------------------------+
|        2|                    2017|               16.6|              3.4|             3.0|                  19.2|                    121.7|
|        1|                    1516|               16.1|              3.3|             2.8|                  14.2|                     94.1|
|        4|                       7|               16.0|              3.4|             2.7|                  14.5|                      0.5|
+---------+------------------------+-------------------+-----------------+----------------+----------------------+-------------------------+

In [9]:
# check for nulls in all columns

df.select([
    F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+----+--------+--------+----+----------+------+------------+---------------+
|vendor_id|month|year|duration|distance|fare|total_cost|extras|payment_type|passenger_count|
+---------+-----+----+--------+--------+----+----------+------+------------+---------------+
|        0|    0|   0|       0|       0|   0|         0|     0|           0|              0|
+---------+-----+----+--------+--------+----+----------+------+------------+---------------+

In [10]:
# various payment type method codes

df.select('payment_type').distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(payment_type='3'), Row(payment_type='1'), Row(payment_type='5'), Row(payment_type='4'), Row(payment_type='2')]

In [11]:
# find most freq payment types and passenger counts (increase support to filter out less frequent items)
df.freqItems(['payment_type', 'passenger_count'], support=0.3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-------------------------+
|payment_type_freqItems|passenger_count_freqItems|
+----------------------+-------------------------+
|             [2, 1, 3]|                [2, 1, 3]|
+----------------------+-------------------------+

In [12]:
# can run common sql queries by registering temp table

df.createOrReplaceTempView("taxis")

# find avergae cost in summer months in 2018
spark.sql("SELECT avg(total_cost) from taxis where month between 6 and 9 and year=2018").show()
# find the total number of trips grouped by months
spark.sql("SELECT count(*) as total from taxis group by month order by total desc").show()

spark.catalog.dropTempView("taxis")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|   avg(total_cost)|
+------------------+
|16.669653451969616|
+------------------+

+--------+
|   total|
+--------+
|19724073|
|19351484|
|19326215|
|18589727|
|18470880|
|18370716|
|17681132|
|17661313|
|17428111|
|16985499|
|16438063|
|16271231|
+--------+

In [13]:
# historgram of earnings 

df.select('total_cost').rdd.flatMap(lambda x: x).histogram(10)[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.0, 90707.10625, 181414.2125, 272121.31875, 362828.425, 453535.53125, 544242.6375, 634949.74375, 725656.85, 816363.9562499999, 907071.0625]

In [52]:
df.groupby('month')\
.agg(F.round(F.avg('total_cost'), 2)\
     .alias('avg_cost'), (F.sum('total_cost')/1000000).cast(IntegerType())\
     .alias('total_cost (millions)'), F.round(F.avg('extras'),2)\
     .alias('avg_extras')).orderBy(F.col('total_cost (millions)').desc(), F.col('avg_cost').desc(),F.col('avg_extras').desc())\
.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+---------------------+----------+
|month|avg_cost|total_cost (millions)|avg_extras|
+-----+--------+---------------------+----------+
|    5|    16.8|                  324|      3.39|
|    3|   16.11|                  317|      3.31|
|    4|   16.36|                  316|      3.31|
|   10|   16.86|                  313|      3.41|
|    6|   16.67|                  306|      3.38|
|   11|   16.65|                  289|      3.37|
|   12|   16.36|                  289|      3.29|
|    1|   15.52|                  286|       3.2|
|    9|   16.77|                  284|      3.37|
|    2|   15.57|                  274|      3.22|
|    7|   16.49|                  270|      3.29|
|    8|   16.56|                  269|      3.32|
+-----+--------+---------------------+----------+

In [57]:
df.filter()groupby('vendor_id','passenger_count', 'payment_type').agg(F.sum(F.col('fare')).alias('Fare paid')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+------------+--------------------+
|vendor_id|passenger_count|payment_type|           Fare paid|
+---------+---------------+------------+--------------------+
|        2|              5|           2|3.7412129330002226E7|
|        1|              6|           1|   362442.4100148771|
|        1|              5|           3|   17653.72000234574|
|        1|              3|           3|   552109.7400273383|
|        1|              9|           2|               666.5|
|        1|              0|           2|  3905156.3101407494|
|        1|              5|           1|  522941.74000841007|
|        2|              1|           3|                 0.0|
|        1|              4|           3|  407910.40999061987|
|        2|              0|           2|    35366.4299970828|
|        1|              0|           4|   77041.15999911726|
|        2|              2|           1|1.7761604253027302E8|
|        1|              4|           1|1.2494099099999025E7|
|       